# Library Testing! 

Start by testing interpolation against existing UR3 library

1. Import + initialize Liam and Aengus libray 

In [1]:
from myur import MyUR3e

# Create an instance of the MyUR3e class
robot = MyUR3e()
print("done")

done


2. Import Hannah and Leia library + create a test path

In [3]:
import UR_lib
import time
import numpy as np

# Test path
coordinates_path = [[0,0,0,0,0,0],
                    [3,4,1,30,0,0],
                    [6,8,2,90,20,0],
                    [6,8,2,90,20,20]]
print("done")

done


You can check the nodes by running:

In [4]:
!ros2 node list

/GripperNode
/controller_manager
/controller_stopper
/dashboard_client
/force_torque_sensor_broadcaster
/forward_position_controller
/gripper_client
/gripper_client
/io_and_status_controller
/joint_state_broadcaster
/joint_state_subscriber
/joint_state_subscriber
/remote_control_client
/remote_control_client
/robot_state_publisher
/scaled_joint_trajectory_controller
/speed_scaling_state_broadcaster
/ur_configuration_controller
/ur_dashboard_client
/ur_dashboard_client
/urscript_interface
/wrench_subscriber
/wrench_subscriber


This should be a working path that throws no errors and prints results. 

In [8]:
interpolated_result = UR_lib.linear_interp(coordinates_path, step_size = 1, debugging=True, aligned=False)


--- Original Coordinates ---
[0, 0, 0, 0, 0, 0]
[3, 4, 1, 30, 0, 0]
[6, 8, 2, 90, 20, 0]
[6, 8, 2, 90, 20, 20]

--- Interpolated Coordinates ---
Point 0: [0, 0, 0, 0, 0, 0]
Point 1: [0.6000000000000001, 0.8, 0.2, 6.0, 0.0, 0.0]
Point 2: [1.2000000000000002, 1.6, 0.4, 12.0, 0.0, 0.0]
Point 3: [1.7999999999999998, 2.4, 0.6, 18.0, 0.0, 0.0]
Point 4: [2.4000000000000004, 3.2, 0.8, 24.0, 0.0, 0.0]
Point 5: [3.0, 4.0, 1.0, 30.0, 0.0, 0.0]
Point 6: [3.6, 4.8, 1.2, 42.0, 4.0, 0.0]
Point 7: [4.2, 5.6, 1.4, 54.0, 8.0, 0.0]
Point 8: [4.8, 6.4, 1.6, 66.0, 12.0, 0.0]
Point 9: [5.4, 7.2, 1.8, 78.0, 16.0, 0.0]
Point 10: [6.0, 8.0, 2.0, 90.0, 20.0, 0.0]
Point 11: [6.0, 8.0, 2.0, 90.0, 20.0, 20.0]
Total interpolated points: 12


This path should throw no errors but no interpolation should occur (Original = Interpolated)

In [4]:
# nothing should happen here
zeros_path = [[0,0,0,0,0,0],
            [0,0,0,0,0,40],
            [0,0,0,0,0,0]]
UR_lib.linear_interp(zeros_path, step_size=1, debugging=True, aligned=True)


--- Original Coordinates ---
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 40]
[0, 0, 0, 0, 0, 0]

--- Interpolated Coordinates ---
Point 0: [0, 0, 0, 0, 0, 180.0]
Point 1: [0.0, 0.0, 0.0, 0.0, 0.0, 180.0]
Point 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Total interpolated points: 3


[[0, 0, 0, 0, 0, 180.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 180.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

All of the following should throw errors for invalid input. 

In [11]:
# Error Testing
no_path = []
path_too_short = [[0,0,0,0,0,0]]

UR_lib.linear_interp(no_path, step_size=1, debugging=True)
UR_lib.linear_interp(zeros_path, step_size=0, debugging=True)
UR_lib.linear_interp(path_too_short, step_size=1, debugging=True)

Error: coordinate list invalid. Length is 0. Ensure at least 2 points to interpolate
Error: step size must be greater than 0.
Error: coordinate list invalid. Length is 1. Ensure at least 2 points to interpolate


[[0, 0, 0, 0, 0, 0]]

## Relative Trajectory testing: 

In [5]:
# Testing relative trajectory
relative_trajectory = np.array([
    [0.0, 0.0, -0.01, 0.0, 0.0, 0.0],
    [0.1, 0.0, 0.0, 0.0, 0.0, 90.0],
])
# relative_trajectory = linear_interp(relative_trajectory, debugging=True)
# Move the end effector through the trajectory
start = [0.2, 0.1, 0.4, 0.0, 0.0, 0.0]
global_trajectory = UR_lib.relative_to_global(relative_trajectory, start)

print(f"global: {global_trajectory}")

global: [[0.2, 0.1, 0.4, 0.0, 0.0, 0.0], [0.2, 0.1, 0.39, 0.0, 0.0, 0.0], [0.30000000000000004, 0.1, 0.39, 0.0, 0.0, 90.0]]


## Testing with the arm: 

In [5]:
# Move in a 10cm square using global relative commands
relative_trajectory = [
    [0.0, 0.0, -0.01, 0.0, 0.0, 0.0],
    [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 90.0],
    [0.0, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, -90.0],
    [-0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0, 0.0, 0.0, 0.0, 0.0, 90.0],
    [0.0, -0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, -90.0]
]
start_pos = [0.2, 0.1, 0.4, 0.0, 0.0, 0.0]
global_trajectory = UR_lib.relative_to_global(relative_trajectory, start_pos)
global_trajectory = UR_lib.linear_interp(global_trajectory, debugging=True, aligned=False)
# Move the end effector through the trajectory
robot.move_global(start_pos)
robot.move_global(global_trajectory, (len(global_trajectory)/2))
print("done")


--- Original Coordinates ---
[0.2, 0.1, 0.4, 0.0, 0.0, 0.0]
[0.2, 0.1, 0.39, 0.0, 0.0, 0.0]
[0.30000000000000004, 0.1, 0.39, 0.0, 0.0, 0.0]
[0.30000000000000004, 0.1, 0.39, 0.0, 0.0, 90.0]
[0.30000000000000004, 0.2, 0.39, 0.0, 0.0, 90.0]
[0.30000000000000004, 0.2, 0.39, 0.0, 0.0, 0.0]
[0.20000000000000004, 0.2, 0.39, 0.0, 0.0, 0.0]
[0.20000000000000004, 0.2, 0.39, 0.0, 0.0, 90.0]
[0.20000000000000004, 0.1, 0.39, 0.0, 0.0, 90.0]
[0.20000000000000004, 0.1, 0.39, 0.0, 0.0, 0.0]

--- Interpolated Coordinates ---
Point 0: [0.2, 0.1, 0.4, 0.0, 0.0, 0.0]
Point 1: [0.2, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 2: [0.21000000000000002, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 3: [0.22000000000000003, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 4: [0.23, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 5: [0.24000000000000002, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 6: [0.25, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 7: [0.26, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 8: [0.27, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 9: [0.28, 0.1, 0.39, 0.0, 0.0, 0.0]
Point 10: [0.2900000000

[INFO] [1750169397.777470004] [remote_control_client]: Goal #1: Executing
[INFO] [1750169402.863243425] [remote_control_client]: Goal #1: Completed
[INFO] [1750169402.929959302] [remote_control_client]: Goal #2: Executing


done


[INFO] [1750169430.985033851] [remote_control_client]: Goal #2: Completed


## Drawing Circles

In [15]:
# center = [0.4, 0.2, 0.5, 0, 0, 0] # generated location
center = [0.25, 0.1, 0.34, 0.0, 0.0, 0.0]
robot.move_global(center)
print("make sure center position seems viable.")
time.sleep(2)
trajectory = UR_lib.draw_circle(
    center,
    radius=0.1,
    num_points=24,
    degrees=True,
    plane_rotation=[90, 0, 0],  # Tilt circle 45° around Y-axis
    debugging=True,
    aligned=False
)
robot.move_global(trajectory, len(trajectory)/2)
print("done")

[INFO] [1749829310.168690231] [remote_control_client]: Goal #24: Executing


make sure center position seems viable.


[INFO] [1749829315.223122020] [remote_control_client]: Goal #24: Completed


Theta 345.0° -> [np.float64(0.3465925826289068), np.float64(0.1), np.float64(0.3141180954897479), 0.1, 0.1, 0.1]


[INFO] [1749829317.393285459] [remote_control_client]: Goal #25: Executing


done


[INFO] [1749829334.450390998] [remote_control_client]: Goal #25: Completed


## Zig Zag

In [6]:
robot.read_global_pos()

[0.28750181198120117,
 -0.006999377626925707,
 0.16555452346801758,
 -0.7003953924492897,
 1.6304775652904702,
 -5.704603487096954]

In [29]:
zig_zag = [[.26, -0.02, 0, 0, 0, 0],
           [.2, -0.01, 0, 0, 0, 0],
           [.26, 0, 0, 0, 0, 0],
           [.2, 0.01, 0, 0, 0, 0],
           [.26, 0.02, 0, 0, 0, 0],
           [.2, 0.03, 0, 0, 0, 0],
           [.26, 0.04, 0, 0, 0, 0],
           [.2, 0.03, 0, 0, 0, 0],
           [.26, 0.02, 0, 0, 0, 0],
           [.2, 0.01, 0, 0, 0, 0],
           [.26, 0, 0, 0, 0, 0],
           [.2, -0.01, 0, 0, 0, 0],
           [.26, -0.02, 0, 0, 0, 0]]
UR_lib.set_z(zig_zag, .175)
start = [.28, -.02, .3, 0, 0, 0]
robot.move_global(start)
time.sleep(2)
zig_zag = UR_lib.linear_interp(zig_zag, debugging=False)
robot.move_global(zig_zag, 15, wait=True)
robot.move_global([.26, -0.02, .2, 0, 0, 0])
print("done")

[INFO] [1750190947.666781986] [remote_control_client]: Goal #61: Executing
[INFO] [1750190952.722904264] [remote_control_client]: Goal #61: Completed


done
